In [13]:
import pandas as pd
import numpy as np
import time

In [14]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

                 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
options.add_argument('--window-size=1920,1080')


<h1> 일별 데이터 </h1>

In [15]:
# 1. 서울시 지하철 호선별 역별 승하차 인원 정보
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do")
tbody = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="AXGridTarget_AX_tbody"]')))

# 2. 검색 필드 선택
select_element = driver.find_element(By.NAME, 'filterCol')
select = Select(select_element)
select.select_by_value('LINE_NUM')

# 3. 검색어 입력
search = driver.find_element(By.NAME, 'txtFilter')
submit = driver.find_element(By.XPATH, '//*[@id="frmSheet"]/div[3]/button[3]/span')
ActionChains(driver).send_keys_to_element(search, "3호선").perform()
time.sleep(1)
ActionChains(driver).click(submit).perform()
time.sleep(1)
ActionChains(driver).scroll_by_amount(0, 200).perform()

# 4. 테이블 스크롤 내리고, 데이터 저장
mouse_tracker = driver.find_element(By.XPATH, '//*[@id="AXGridTarget_AX_scrollYHandle"]')
columns = ['date', 'line', 'station', 'in', 'out', 'upload_date']
res = pd.DataFrame(columns=columns)
scrollbar = driver.find_element(By.XPATH, '//*[@id="AXGridTarget_AX_scrollTrackY"]')
scroll_height = scrollbar.size['height']
y = driver.find_element(By.XPATH, '//*[@id="AXGridTarget_AX_scrollYHandle"]').size['height'] - 7
while y < scroll_height:
    ActionChains(driver).move_to_element_with_offset(mouse_tracker, 0, 0).click_and_hold().move_by_offset(0,6).perform()
    tbody = driver.find_element(By.XPATH, '//*[@id="AXGridTarget_AX_tbody"]')
    tbody_split = np.array(tbody.text.split('\n')).reshape(-1, 6)
    df = pd.DataFrame(tbody_split, columns=columns)
    res = pd.concat([res, df])
    y += 6
    time.sleep(0.3)

# 5. 데이터 전처리
res = res.drop_duplicates()
res = res.apply(lambda x: x.str.replace(',',''))
res[["in", "out"]] = res[["in", "out"]].apply(pd.to_numeric)
res[["date", "upload_date"]] = res[["date", "upload_date"]].apply(pd.to_datetime)

#### 5월 1일에 데이터 업데이트 되어서 현재는 불일치, 위 로직은 맞아요!
# # 6. 실제 데이터 불러오기
# actual_res = pd.read_csv('subway_daily.csv', encoding='cp949')
# # 7. 데이터 검증
# def test_subway():
#     assert actual_res['승차총승객수'][:1000].sum() == res['in'].sum(), '승차인원 데이터 불일치'
#     assert actual_res['하차총승객수'][:1000].sum() == res['out'].sum(), '하차인원 데이터 불일치'
#     assert len(res.index) == 1000, '데이터 수 불일치'
#     print('테스트 통과')
# test_subway()


In [16]:
res.head()

,date,line,station,in,out,upload_date
0,2023-04-28,3호선,가락시장,10668,10525,2023-05-01
1,2023-04-28,3호선,경복궁(정부서울청사),31604,32504,2023-05-01
2,2023-04-28,3호선,경찰병원,8700,8711,2023-05-01
3,2023-04-28,3호선,고속터미널,58992,60829,2023-05-01
4,2023-04-28,3호선,교대(법원.검찰청),20390,11932,2023-05-01


In [17]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 17
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1000 non-null   datetime64[ns]
 1   line         1000 non-null   object        
 2   station      1000 non-null   object        
 3   in           1000 non-null   int64         
 4   out          1000 non-null   int64         
 5   upload_date  1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 54.7+ KB


<h1>시간별 데이터</h1>

In [18]:
# 1. 서울시 지하철 호선별 역별 승하차 인원 정보
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do")
tbody = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="AXGridTarget_AX_tbody"]')))

# 2. 검색 필드 선택
select_element = driver.find_element(By.NAME, 'filterCol')
select = Select(select_element)
select.select_by_value('LINE_NUM')

# 3. 검색어 입력
search = driver.find_element(By.NAME, 'txtFilter')
submit = driver.find_element(By.XPATH, '//*[@id="frmSheet"]/div[3]/button[3]/span')
ActionChains(driver).send_keys_to_element(search, "3호선").perform()
time.sleep(1)
ActionChains(driver).click(submit).perform()
time.sleep(1)
ActionChains(driver).scroll_by_amount(0, 200).perform()

# 4. 데이터 수집
current_date = pd.Timestamp.now()
res = pd.DataFrame()
mouse_tracker = driver.find_element(By.XPATH, '//*[@id="AXGridTarget_AX_scrollYHandle"]')

while True:
    scroll = driver.find_element(By.XPATH, '//*[@id="AXGridTarget_AX_scrollYHandle"]')
    ActionChains(driver).move_to_element_with_offset(mouse_tracker, 0, 0).click_and_hold().move_by_offset(0,6).perform()

    html = driver.page_source
    df = pd.read_html(html)[5].iloc[1:-1,:-1]
    date_of_data  = pd.to_datetime(df[0].iloc[-1], format='%Y%m')

    res = pd.concat([res, df])
    if current_date.year*12+current_date.month-(date_of_data.year*12+date_of_data.month) > 3:
        break
    time.sleep(0.2)

# 5. 데이터 전처리
res = res.drop_duplicates()
res = res.apply(lambda x: x.str.replace(',','') if x.dtype == 'str' else x)

columns = pd.read_html(html)[4]
columns = columns.squeeze().tolist()
columns = list(map(lambda x: x.replace('T', ''), columns))
res.columns = columns

res["사용월"] = pd.to_datetime(res["사용월"], format='%Y%m')
res["작업일자"] = pd.to_datetime(res["작업일자"], format='%Y%m%d')
res = res.apply(lambda x: x.astype(int) if x.dtype == 'float64' else x)
res = res.loc[res['사용월'] > current_date - pd.DateOffset(months=3)]

# # 6. 실제 데이터 불러오기
# actual_res = pd.read_csv('subway_hourly.csv', encoding='cp949')
# actual_res["사용월"] = pd.to_datetime(actual_res["사용월"], format='%Y%m')
# actual_res = actual_res.loc[ actual_res['사용월'] >= current_date-pd.DateOffset(months=3) ]

# # 7. 데이터 검증
# def test_subway():
#     assert actual_res["04시-05시 승차인원"].sum() == res['04시-05시 승차인원'].sum(), '승차인원 데이터 불일치'
#     assert actual_res["04시-05시 하차인원"].sum() == res['04시-05시 하차인원'].sum(), '하차인원 데이터 불일치'
#     assert len(res.index) == len(actual_res.index), '데이터 수 불일치'
#     print('테스트 통과')
# test_subway()

In [19]:
res.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
1,2023-03-01,3호선,가락시장,46,8,3058,485,7682,6151,22601,...,5871,374,1963,2,42,0,0,0,0,2023-04-03
2,2023-03-01,3호선,경복궁(정부서울청사),56,0,2702,1722,6466,14615,15350,...,5861,1012,1657,1,48,1,0,0,0,2023-04-03
3,2023-03-01,3호선,경찰병원,18,0,2955,325,8218,3159,25304,...,5952,162,1642,1,0,0,0,0,0,2023-04-03
4,2023-03-01,3호선,고속터미널,92,6,4853,3852,7065,34531,28216,...,12485,2034,2279,0,0,0,0,0,0,2023-04-03
5,2023-03-01,3호선,교대(법원.검찰청),80,5,1757,654,4170,7850,10082,...,2668,1378,835,0,1,0,1,0,0,2023-04-03


In [20]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 1 to 24
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   사용월           34 non-null     datetime64[ns]
 1   호선명           34 non-null     object        
 2   지하철역          34 non-null     object        
 3   04시-05시 승차인원  34 non-null     int64         
 4   04시-05시 하차인원  34 non-null     int64         
 5   05시-06시 승차인원  34 non-null     int64         
 6   05시-06시 하차인원  34 non-null     int64         
 7   06시-07시 승차인원  34 non-null     int64         
 8   06시-07시 하차인원  34 non-null     int64         
 9   07시-08시 승차인원  34 non-null     int64         
 10  07시-08시 하차인원  34 non-null     int64         
 11  08시-09시 승차인원  34 non-null     int64         
 12  08시-09시 하차인원  34 non-null     int64         
 13  09시-10시 승차인원  34 non-null     int64         
 14  09시-10시 하차인원  34 non-null     int64         
 15  10시-11시 승차인원  34 non-null     int64         
 1